In [1]:
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from scipy.stats import spearmanr

# Корпус
1020 отрывков из пяти слов, начинающихся на слово "суд", лемматизированных и без стоп-слов.

In [2]:
with open('court-V-N.csv','r',encoding='utf-8') as f:
    corp = [x.strip().split(' ,') for x in f.readlines()]
print('N tokens:',sum([len(x) for x in corp]))

N tokens: 5100


# Золотой стандарт
Интуиция + прикидки частотности биграммы и составляющих по корпусу на глаз.

In [3]:
golden_colloc = [
    ('ПРИНЯТЬ', 'РЕШЕНИЕ'),
    ('МЕРА', 'ПРЕСЕЧЕНИЕ'),
    ('САНКЦИЯ', 'АРЕСТ'),
    ('УДОВЛЕТВОРИТЬ', 'ИСК'),
    ('УДОВЛЕТВОРИТЬ', 'ХОДАТАЙСТВО'),
    ('ПРИЗНАТЬ', 'ВИНОВНАЯ'),
    ('ВЫНЕСТИ', 'РЕШЕНИЕ'),
    ('ОТКАЗАТЬ', 'УДОВЛЕТВОРЕНИЕ'),
    ('НАЛОЖИТЬ', 'АРЕСТ'),
    ('ОСТАВИТЬ', 'СИЛА')
]

# Автоматическое извлечение коллокаций
### Метрики
* Likelihood ratio
* PMI

In [4]:
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(corp)

## Likelihood ratio

In [5]:
lr_colloc = finder.score_ngrams(bigram_measures.likelihood_ratio)
lr_all_ranks = {x[0]:i for i,x in enumerate(lr_colloc)}
lr_colloc[:10]

[(('СУД', 'ПРИЗНАТЬ'), 219.22906047906903),
 (('ВЫДАТЬ', 'САНКЦИЯ'), 134.79222855631895),
 (('СУД', 'УДОВЛЕТВОРИТЬ'), 132.5976240279905),
 (('УДОВЛЕТВОРИТЬ', 'ИСК'), 127.52283038813364),
 (('УДОВЛЕТВОРИТЬ', 'ХОДАТАЙСТВО'), 126.30257129727798),
 (('ПРИНЯТЬ', 'РЕШЕНИЕ'), 124.73943459343414),
 (('САНКЦИЯ', 'АРЕСТ'), 116.1065067993143),
 (('МЕРА', 'ПРЕСЕЧЕНИЕ'), 101.98507334340232),
 (('НАЛОЖИТЬ', 'АРЕСТ'), 96.03598326740575),
 (('СУД', 'ВЫНЕСТИ'), 88.1625667221642)]

Из 10 выделенных по LR коллокаций 6 есть в ЗС, а три -- начинающиеся со слова "суд", то есть являющиеся сочетанием субъект+глагол -- не знаю насколько можно считать коллокацией вообще. Для исключения таких "коллокаций" можно либо из итогового списка выкинуть все коллокации вида "суд"+глагол, либо из исходного корпуса удалить первое слово каждого предложения (собственно, "суд"). Одна коллокация -- 'ВЫДАТЬ', 'САНКЦИЯ' -- хорошая, ее можно было бы включить в ЗС. 

## PMI
Без ограничений на минимальную частоту работает плохо, выставила частоту 5 (минимальная частота 6 дает такие же результаты, а начиная с частоты 7 не все коллокации из ЗС есть в выделенных по PMI)

In [6]:
finder.apply_freq_filter(5)
pmi_colloc = finder.score_ngrams(bigram_measures.pmi)
pmi_all_ranks = {x[0]:i for i,x in enumerate(pmi_colloc)}
pmi_colloc[:10]

[(('ОСВОБОДИТЬ', 'ЗАЛОГ'), 9.28653418835217),
 (('ЛИШЕНИЕ', 'СВОБОДА'), 9.245892203854822),
 (('ПРОЙТИ', 'ПРЕНИЕ'), 9.053247125912428),
 (('МЕРА', 'ПРЕСЕЧЕНИЕ'), 8.61584181360513),
 (('ИЗБРАТЬ', 'МЕРА'), 8.352807407771335),
 (('ВЫДАТЬ', 'САНКЦИЯ'), 7.536940985858216),
 (('ОСТАВИТЬ', 'СИЛА'), 7.4093909361377035),
 (('АРЕСТОВАТЬ', 'АКЦИЯ'), 7.271887412387769),
 (('СОГЛАСИТЬСЯ', 'ДОВОД'), 7.146356530303909),
 (('НАЧАТЬСЯ', 'РАССМОТРЕНИЕ'), 6.83085470457598)]

В этом топ-10 есть всего 2 коллокации из ЗС ('МЕРА', 'ПРЕСЕЧЕНИЕ' и 'ОСТАВИТЬ', 'СИЛА'). Это менее частотные коллокации, тогда как я выделяла более частотные, поэтому такой маленький процент пересечения понятен. Ошибкой можно назвать только одну коллокацию, опять же, глагол+субъект - 'НАЧАТЬСЯ', 'РАССМОТРЕНИЕ', остальные вполне похожи на коллокации.

# Корреляции

In [7]:
golden_ranks = list(range(1,11))
lr_ranks = [lr_all_ranks[x] for x in golden_colloc]
pmi_ranks = [pmi_all_ranks[x] for x in golden_colloc]

In [8]:
print('LR & PMI')
print(spearmanr(lr_ranks,pmi_ranks))
print()
print('LR & GS')
print(spearmanr(lr_ranks,golden_ranks))
print()
print('PMI & GS')
print(spearmanr(pmi_ranks,golden_ranks))

LR & PMI
SpearmanrResult(correlation=-0.39393939393939392, pvalue=0.25999776683488757)

LR & GS
SpearmanrResult(correlation=0.73333333333333317, pvalue=0.015800596250571581)

PMI & GS
SpearmanrResult(correlation=-0.23636363636363633, pvalue=0.51088531751520017)


Disclaimer: p-value, вероятно, в данном случае верить нельзя, потому что в документации написано: "The p-values are not entirely reliable but are probably reasonable for datasets larger than 500 or so.". А у нас всего 10 объектов.

Между LR и PMI наблюдается небольшая негативная корреляция, то есть чем выше по рангу коллокация по LR, тем она ниже по PMI. Но корреляция все равно не очень большая. Между LR и ЗС зато есть довольно высокая положительная корреляция, соответственно между PMI и ЗС - небольшая отрицательная. Вообще по топ-10 PMI и по этим результатам видно, что PMI при минимальной частоте 5 выделяет коллокации другого порядка. Думаю, что при минимальной частоте >6 результат был бы лучше, но так выделяются не все коллокации из ЗС.